In [2]:
# Question 7

In [27]:
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import quad

In [29]:
# QUESTION 7

For question 7, we know that the azeotrope column and the stripper are both operating at VLE. Because of this we can use Raoult's Law or just the Antoine equation 

In [31]:
# knowns

x1 = 0.9795 # ethanol
x2 = 0.0205 # water
P = 2.0265 # bar, system pressure in stripper
#x2 = 1-x1

In [33]:
def psat(t,i):
    if i == 1:  # ethanol
        A = 8.13484	
        B = 1662.48	
        C = 238.131
     
    elif i == 2:  # water
        A = 8.05573	
        B = 1723.64	
        C = 233.076
     

        # Calculate saturation pressure using Antoine equation
    P_sat = 10 ** (A - (B/ (t + C))) / 750.062 # convert to bar
    return(P_sat)  # in bar

psat(100,2)

1.0132527754404501

In [35]:
# function for fsolve for stripper

def f(a):
    eq1 =  P - psat(a,2)
    return(eq1)

In [37]:
guesses = 160
f(guesses)

-4.219880124915525

In [39]:
ans = fsolve(f, guesses)
TF = ans[0] * 9/5 + 32

In [41]:
print('t =', round(ans[0], 4), 'Celsius')
print('t =', round(TF, 4), 'Farenheit')
# for the stripper

t = 120.572 Celsius
t = 249.0296 Farenheit


In [44]:
# function for fsolve for azeotrope column
Pac = 3.34373 # bar, system pressure for azeotrope column

def f1(a):
    t, y1 = a
    eq1 = x1 * psat(t,1) - y1 * Pac
    eq2 = x2 * psat(t,2) - (1-y1) * Pac
    return([eq1,eq2])

In [46]:
guesses1 = [200,0.8]
f1(guesses1)

[25.918410589219683, -0.3433648243825195]

In [48]:
ans1 = fsolve(f1, guesses1)
TF1 = ans1[0] * 9/5 + 32

In [50]:
print('t =', round(ans1[0], 4), 'Celsius')
print('t =', round(TF1, 4), 'Farenheit')
# for the azeotrope column

t = 113.2854 Celsius
t = 235.9137 Farenheit


In [54]:
# Question 9

In [56]:
# minimum pressure at which cooking vessel must operate
# sat. pressure at 320 F = 160 C

def psat(t,i):
    if i == 2:  # water
        A = 8.05573	
        B = 1723.64	
        C = 233.076
     # Calculate saturation pressure using Antoine equation
    P_sat = 10 ** (A - (B/ (t + C))) / 750.062 # convert to bar
    return(P_sat)

min_pressure = psat(160, 2)
print('min_pressure =', round(min_pressure, 4), 'bar')

min_pressure = 6.2464 bar


In [58]:
# pressure to which mash must be flashed to provide a resultant liquor temp of 145 F = 62.7778 C

flash_pressure = psat(62.778, 2)
print('flash_pressure =', round(flash_pressure, 4), 'bar')

flash_pressure = 0.2263 bar


In [60]:
m_corn = 138600 # lbm/hr corn meal
m_h2o = 360533 # lbm/hr total water coming in to mixing tank

m_mash = m_corn + m_h2o
W_corn = m_corn/m_mash
W_h2o = m_h2o/m_mash

print(f"m_mash = {m_mash :.6f} lbm/hr flow rate of mash")
print(f"W_corn = {W_corn :.6f} weight % of corn solids in mash")
print(f"W_h2o = {W_h2o :.6f} weight % of water in mash")

m_mash = 499133.000000 lbm/hr flow rate of mash
W_corn = 0.277681 weight % of corn solids in mash
W_h2o = 0.722319 weight % of water in mash


In [62]:
# for precook vessel

pv_m_mashin = m_mash * W_corn # mass flowrate of mash solids into precook vessel
pv_m_h2oin = m_mash * W_h2o # mass flowrate of water into precook vessel
pv_m_mashout = pv_m_mashin # mass flowrate of mash solids OUT of precook vessel

tin_mash1 = 37.7778 # 100 F in C
tin_steam1 = 121 # in C, Tsat temp from 15 psig steam
tout_mash1 = 62.7778 # 145 F in C

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmash1 = quad(Cpmash, tin_mash1, tout_mash1)[0]
delta_hmash_jPg1 = delta_hmash1/MWh2o
delta_hmash_jPlbm1 = delta_hmash_jPg1/lbmPg

print(f"delta_hmash1 = {delta_hmash_jPlbm1 :.6f} J/lbm = enthalpy change needed for mash")

delta_hmash1 = 47485.600905 J/lbm = enthalpy change needed for mash


In [64]:
# calculating delta hs and hs for energy balance around precook vessel


def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to precook vessel 
h_watermash_in1 = quad(Cph2o, ref, tin_mash1)[0]

# calculating h2, enthalpy of water vapor coming in to precook vessel
deltahvap = 39651 # @ 121 C, J/mol roughly from Dippr 
h_watervapor_in1 = quad(Cph2o, ref, tin_steam1)[0] + deltahvap

# calculating h3, enthalpy of water coming out of precook vessel in mash
h_watermash_out1 = quad(Cph2o, ref, tout_mash1)[0]


In [66]:
h_watermash_in_jPg1 = h_watermash_in1/MWh2o
h_watermash_in_jPlbm1 = h_watermash_in_jPg1/lbmPg

print(f"h_watermash_in_jPlbm1 = {h_watermash_in_jPlbm1 :.6f} J/lbm = enthalpy water coming in to precook vessel")

h_watervapor_in_jPg1 = h_watervapor_in1/MWh2o
h_watervapor_in_jPlbm1 = h_watervapor_in_jPg1/lbmPg

print(f"h_watervapor_in_jPlbm1 = {h_watervapor_in_jPlbm1 :.6f} J/lbm = enthalpy water vapor coming in precook vessel")

h_watermash_out_jPg1 = h_watermash_out1/MWh2o
h_watermash_out_jPlbm1 = h_watermash_out_jPg1/lbmPg

print(f"h_watermash_out_jPlbm1 = {h_watermash_out_jPlbm1 :.6f} J/lbm = enthalpy water coming out in mash")

h_watermash_in_jPlbm1 = 71756.061354 J/lbm = enthalpy water coming in to precook vessel
h_watervapor_in_jPlbm1 = 1228690.553192 J/lbm = enthalpy water vapor coming in precook vessel
h_watermash_out_jPlbm1 = 119241.662259 J/lbm = enthalpy water coming out in mash


In [68]:
# energy balance for precook vessel

def f_1(a):
    m_h2ovaporin1, m_h2omash_out1 = a # unknowns, water coming in as vapor and water coming out in mash
    
    eq1 = pv_m_h2oin + m_h2ovaporin1 - m_h2omash_out1
    eq2 = pv_m_mashin * delta_hmash_jPlbm1 + m_h2omash_out1 * h_watermash_out_jPlbm1 - m_h2ovaporin1 * h_watervapor_in_jPlbm1 - pv_m_h2oin * h_watermash_in_jPlbm1
    return([eq1,eq2])

guess_1 = [50000, 500000]
ans1 = fsolve(f_1, guess_1)
m_h2ovaporin1 = ans1[0]
m_h2omash_out1 = ans1[1]
print(f"m_h2ovaporin1 = {m_h2ovaporin1 :.6f} lbm/hr = flowrate of vapor in to precook Vessel")
print(f"m_h2omash_out1 = {m_h2omash_out1 :.6f} lbm/hr = flowrate of water in mash out of precook Vessel")


m_h2ovaporin1 = 21363.427040 lbm/hr = flowrate of vapor in to precook Vessel
m_h2omash_out1 = 381896.427040 lbm/hr = flowrate of water in mash out of precook Vessel


In [70]:
m_mashnew1 = pv_m_mashout + m_h2omash_out1  # total flowrate (mash + water) coming out of precook vessel
W_mash1 = pv_m_mashout/m_mashnew1
W_h2o1 = m_h2omash_out1/m_mashnew1

print(f"W_mash1 = {W_mash1 :.6f} new weight % of corn solids in mash out of precook vessel")
print(f"W_h2o1 = {W_h2o1 :.6f} new weight % of water in mash out of precook vessel")
print(f"m_mashnew1 = {m_mashnew1 :.6f} lbm/hr = flowrate of mash and water out of precook vessel")

W_mash1 = 0.266284 new weight % of corn solids in mash out of precook vessel
W_h2o1 = 0.733716 new weight % of water in mash out of precook vessel
m_mashnew1 = 520496.427040 lbm/hr = flowrate of mash and water out of precook vessel


In [72]:
# heat exchanger drives mash from 145 F to 230 F see question 10

In [74]:
# heater from 230 F to 320 F

# for heater 

heater_m_mashin = m_mashnew1 * W_mash1 # mass flowrate of mash solids into heater
heater_m_h2oin = m_mashnew1 * W_h2o1 # mass flowrate of water into heater
heater_m_mashout = heater_m_mashin # mass flowrate of mash solids OUT of heater

tin_mash2 = 110 # 230 F in C
tin_steam2 = 185 # in C, Tsat temp from 150 psig steam
tout_mash2 = 160 # 320 F in C

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmash2 = quad(Cpmash, tin_mash2, tout_mash2)[0]
delta_hmash_jPg2 = delta_hmash2/MWh2o
delta_hmash_jPlbm2 = delta_hmash_jPg2/lbmPg

print(f"delta_hmash2 = {delta_hmash_jPlbm2 :.6f} J/lbm = enthalpy change needed for mash in heater")

delta_hmash2 = 94971.201809 J/lbm = enthalpy change needed for mash in heater


In [76]:
# calculating delta hs and hs for energy balance around heater


def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to heater
h_watermash_in2 = quad(Cph2o, ref, tin_mash2)[0]

# calculating h2, enthalpy of water vapor coming in to heater
deltahvap2 = 35939 # @ 185 C, J/mol roughly from Dippr
h_watervapor_in2 = quad(Cph2o, ref, tin_steam2)[0] + deltahvap2

# calculating h3, enthalpy of water coming out of heater in mash
h_watermash_out2 = quad(Cph2o, ref, tout_mash2)[0]


In [78]:
h_watermash_in_jPg2 = h_watermash_in2/MWh2o
h_watermash_in_jPlbm2 = h_watermash_in_jPg2/lbmPg

print(f"h_watermash_in_jPlbm2 = {h_watermash_in_jPlbm2 :.6f} J/lbm = enthalpy water coming in to heater")

h_watervapor_in_jPg2 = h_watervapor_in2/MWh2o
h_watervapor_in_jPlbm2 = h_watervapor_in_jPg2/lbmPg

print(f"h_watervapor_in_jPlbm2 = {h_watervapor_in_jPlbm2 :.6f} J/lbm = enthalpy water vapor coming in heater")

h_watermash_out_jPg2 = h_watermash_out2/MWh2o
h_watermash_out_jPlbm2 = h_watermash_out_jPg2/lbmPg

print(f"h_watermash_out_jPlbm2 = {h_watermash_out_jPlbm2 :.6f} J/lbm = enthalpy water coming out in mash")

h_watermash_in_jPlbm2 = 208936.643981 J/lbm = enthalpy water coming in to heater
h_watervapor_in_jPlbm2 = 1256743.585111 J/lbm = enthalpy water vapor coming in heater
h_watermash_out_jPlbm2 = 303907.845790 J/lbm = enthalpy water coming out in mash


In [80]:
# energy balance for heater

def f_2(a):
    m_h2ovaporin2, m_h2omash_out2 = a # unknowns, water coming in as vapor and water coming out in mash
    
    eq1 = heater_m_h2oin + m_h2ovaporin2 - m_h2omash_out2
    eq2 = heater_m_mashin * delta_hmash_jPlbm2 + m_h2omash_out2 * h_watermash_out_jPlbm2 - m_h2ovaporin2 * h_watervapor_in_jPlbm2 - heater_m_h2oin * h_watermash_in_jPlbm2
    return([eq1,eq2])

guess_2 = [50000, 500000]
ans2 = fsolve(f_2, guess_2)
m_h2ovaporin2 = ans2[0]
m_h2omash_out2 = ans2[1]
print(f"** m_h2ovaporin2 = {m_h2ovaporin2 :.6f} lbm/hr = flowrate of vapor (live steam) in to heater **")
print(f"m_h2omash_out2 = {m_h2omash_out2 :.6f} lbm/hr = flowrate of water in mash out of heater")


** m_h2ovaporin2 = 51879.006185 lbm/hr = flowrate of vapor (live steam) in to heater **
m_h2omash_out2 = 433775.433225 lbm/hr = flowrate of water in mash out of heater


In [82]:
m_mashnew2 = heater_m_mashout + m_h2omash_out2  # total flowrate (mash + water) coming out of heater
W_mash2 = heater_m_mashout/m_mashnew2
W_h2o2 = m_h2omash_out2/m_mashnew2

print(f"W_mash2 = {W_mash2 :.6f} new weight % of corn solids in mash out of heater")
print(f"W_h2o2 = {W_h2o2 :.6f} new weight % of water in mash out of heater")
print(f"m_mashnew2 = {m_mashnew2 :.6f} lbm/hr = flowrate of mash and water out of heater")

W_mash2 = 0.242149 new weight % of corn solids in mash out of heater
W_h2o2 = 0.757851 new weight % of water in mash out of heater
m_mashnew2 = 572375.433225 lbm/hr = flowrate of mash and water out of heater


In [84]:
# after cooking tank, flowrate is still consistent
# solving now around first flash tank at 15 psig

tin_mash3 = 160 # 320 F in C
tout_mash3 = 121 # in C, temp from 15 psig in flash. Used steam tables to figure it out
tout_steam3 = 121 # same as tout_mash3

flash_m_mashin = m_mashnew2 * W_mash2 # mass flowrate of mash solids into flash tank
flash_m_h2oin = m_mashnew2 * W_h2o2 # mass flowrate of water into flash tank
flash_m_mashout = flash_m_mashin # mass flowrate of mash solids OUT of flash tank

print(f"flash_m_h2oin = {flash_m_h2oin :.6f} lbm/hr = flowrate of water coming in to flash")
print(f"flash_m_mashin = {flash_m_mashin :.6f} lbm/hr = flowrate of mash solids coming in to flash")

flash_m_h2oin = 433775.433225 lbm/hr = flowrate of water coming in to flash
flash_m_mashin = 138600.000000 lbm/hr = flowrate of mash solids coming in to flash


In [86]:
# calculating delta hs and hs

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmash3 = quad(Cpmash, tin_mash3, tout_mash3)[0]
delta_hmash_jPg3 = delta_hmash3/MWh2o
delta_hmash_jPlbm3 = delta_hmash_jPg3/lbmPg

print(f"delta_hmash3 = {delta_hmash_jPlbm3 :.6f} J/lbm = enthalpy change of mash in first flash")

delta_hmash3 = -74077.537411 J/lbm = enthalpy change of mash in first flash


In [88]:
# calculating delta hs and hs

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to flash 
h_watermash_in3 = quad(Cph2o, ref, tin_mash3)[0]

# calculating h2, enthalpy of water coming out of flash in vapor
deltahvap3 = 39651 # @ 121 C, J/mol roughly from Dippr (positive since liquid to steam)
h_watervapor_out3 = quad(Cph2o, ref, tout_steam3)[0] + deltahvap3

# calculating h3, enthalpy of water coming out in mash
h_watermash_out3 = quad(Cph2o, ref, tout_mash3)[0]



In [90]:
h_watermash_in_jPg3 = h_watermash_in3/MWh2o
h_watermash_in_jPlbm3 = h_watermash_in_jPg3/lbmPg

print(f"h_watermash_in_jPlbm3 = {h_watermash_in_jPlbm3 :.6f} J/lbm = enthalpy water coming in to flash")

h_watervapor_out_jPg3 = h_watervapor_out3/MWh2o
h_watervapor_out_jPlbm3 = h_watervapor_out_jPg3/lbmPg

print(f"h_watervapor_out_jPlbm3 = {h_watervapor_out_jPlbm3 :.6f} J/lbm = enthalpy water coming out in vapor")

h_watermash_out_jPg3 = h_watermash_out3/MWh2o
h_watermash_out_jPlbm3 = h_watermash_out_jPg3/lbmPg

print(f"h_watermash_out_jPlbm3 = {h_watermash_out_jPlbm3 :.6f} J/lbm = enthalpy water coming out in mash")

h_watermash_in_jPlbm3 = 303907.845790 J/lbm = enthalpy water coming in to flash
h_watervapor_out_jPlbm3 = 1228690.553192 J/lbm = enthalpy water coming out in vapor
h_watermash_out_jPlbm3 = 229830.308379 J/lbm = enthalpy water coming out in mash


In [92]:
def f_3(a):
    m_h2ovaporout3, m_h2omash_out3 = a # unknowns, water coming out as vapor and water coming out in mash
    
    eq1 = flash_m_h2oin - m_h2ovaporout3 - m_h2omash_out3
    eq2 = flash_m_mashin * delta_hmash_jPlbm3 + m_h2omash_out3 * h_watermash_out_jPlbm3 + m_h2ovaporout3 * h_watervapor_out_jPlbm3 - flash_m_h2oin * h_watermash_in_jPlbm3
    return([eq1,eq2])

guess_3 = [50000, 500000]
ans3 = fsolve(f_3, guess_3)
m_h2ovaporout3 = ans3[0]
m_h2omash_out3 = ans3[1]
print(f"** m_h2ovaporout3 = {m_h2ovaporout3 :.6f} lbm/hr = flowrate of vapor out of first flash **")
print(f"m_h2omash_out3 = {m_h2omash_out3 :.6f} lbm/hr = flowrate of water in mash out of first flash")

** m_h2ovaporout3 = 42448.543516 lbm/hr = flowrate of vapor out of first flash **
m_h2omash_out3 = 391326.889710 lbm/hr = flowrate of water in mash out of first flash


In [94]:
m_mashnew3 = flash_m_mashout + m_h2omash_out3  # total flowrate (mash + liq water) coming out of flash
W_mash3 = flash_m_mashout/m_mashnew3
W_h2o3 = m_h2omash_out3/m_mashnew3

print(f"W_mash3 = {W_mash3 :.6f} new weight % of corn solids in mash out of flash")
print(f"W_h2o3 = {W_h2o3 :.6f} new weight % of water in mash out of flash")
print(f"m_mashnew3 = {m_mashnew3 :.6f} lbm/hr = flowrate of mash and liq water out of flash")

W_mash3 = 0.261546 new weight % of corn solids in mash out of flash
W_h2o3 = 0.738454 new weight % of water in mash out of flash
m_mashnew3 = 529926.889710 lbm/hr = flowrate of mash and liq water out of flash


In [96]:
# solving now around flash vacuum tank at 0.2263 bar

tin_mash4 = 121 # in C
tout_mash4 = 62.78 # in C or 145 F, temp from 0.2263 bar in flash vacuum. Used steam tables to figure it out
tout_steam4 = 62.78 # or 145 F same as tout_mash4

flashvac_m_mashin = m_mashnew3 * W_mash3 # mass flowrate of mash solids into flash vacuum
flashvac_m_h2oin = m_mashnew3 * W_h2o3 # mass flowrate of water into flash vacuum
flashvac_m_mashout = flashvac_m_mashin # mass flowrate of mash solids OUT of flash vacuum

print(f"flashvac_m_h2oin = {flashvac_m_h2oin :.6f} lbm/hr = flowrate of water coming in to flash vacuum")
print(f"flashvac_m_mashin = {flashvac_m_mashin :.6f} lbm/hr = flowrate of mash solids coming in to flash vacuum")

flashvac_m_h2oin = 391326.889710 lbm/hr = flowrate of water coming in to flash vacuum
flashvac_m_mashin = 138600.000000 lbm/hr = flowrate of mash solids coming in to flash vacuum


In [98]:
# calculating delta hs and hs

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmash4 = quad(Cpmash, tin_mash4, tout_mash4)[0]
delta_hmash_jPg4 = delta_hmash4/MWh2o
delta_hmash_jPlbm4 = delta_hmash_jPg4/lbmPg

print(f"delta_hmash4 = {delta_hmash_jPlbm4 :.6f} J/lbm = enthalpy change of mash in flash vacuum")

delta_hmash4 = -110584.467387 J/lbm = enthalpy change of mash in flash vacuum


In [100]:
# calculating delta hs and hs

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to flash vacuum
h_watermash_in4 = quad(Cph2o, ref, tin_mash4)[0]

# calculating h2, enthalpy of water coming out of flash vacuum in vapor
deltahvap4 = 42395 # @ 62.78 C, J/mol roughly from Dippr (positive since liquid to steam)
h_watervapor_out4 = quad(Cph2o, ref, tout_steam4)[0] + deltahvap4

# calculating h3, enthalpy of water coming out in mash
h_watermash_out4 = quad(Cph2o, ref, tout_mash4)[0]


In [102]:
h_watermash_in_jPg4 = h_watermash_in4/MWh2o
h_watermash_in_jPlbm4 = h_watermash_in_jPg4/lbmPg

print(f"h_watermash_in_jPlbm4 = {h_watermash_in_jPlbm4 :.6f} J/lbm = enthalpy water coming in to flash vac")

h_watervapor_out_jPg4 = h_watervapor_out4/MWh2o
h_watervapor_out_jPlbm4 = h_watervapor_out_jPg4/lbmPg

print(f"h_watervapor_out_jPlbm4 = {h_watervapor_out_jPlbm4 :.6f} J/lbm = enthalpy water coming out of flash vac in vapor")

h_watermash_out_jPg4 = h_watermash_out4/MWh2o
h_watermash_out_jPlbm4 = h_watermash_out_jPg4/lbmPg

print(f"h_watermash_out_jPlbm4 = {h_watermash_out_jPlbm4 :.6f} J/lbm = enthalpy water coming out in mash")

h_watermash_in_jPlbm4 = 229830.308379 J/lbm = enthalpy water coming in to flash vac
h_watervapor_out_jPlbm4 = 1187231.013594 J/lbm = enthalpy water coming out of flash vac in vapor
h_watermash_out_jPlbm4 = 119245.840992 J/lbm = enthalpy water coming out in mash


In [104]:
def f_4(a):
    m_h2ovaporout4, m_h2omash_out4 = a # unknowns, water coming out as vapor and water coming out in mash
    
    eq1 = flashvac_m_h2oin - m_h2ovaporout4 - m_h2omash_out4
    eq2 = flashvac_m_mashin * delta_hmash_jPlbm4 + m_h2omash_out4 * h_watermash_out_jPlbm4 + m_h2ovaporout4 * h_watervapor_out_jPlbm4 - flashvac_m_h2oin * h_watermash_in_jPlbm4
    return([eq1,eq2])

guess_4 = [50000, 500000]
ans4 = fsolve(f_4, guess_4)
m_h2ovaporout4 = ans4[0]
m_h2omash_out4 = ans4[1]
print(f"** m_h2ovaporout4 = {m_h2ovaporout4 :.6f} lbm/hr = flowrate of vapor out of flash vac**")
print(f"m_h2omash_out4 = {m_h2omash_out4 :.6f} lbm/hr = flowrate of water in mash out of flash vac")

** m_h2ovaporout4 = 54871.251358 lbm/hr = flowrate of vapor out of flash vac**
m_h2omash_out4 = 336455.638352 lbm/hr = flowrate of water in mash out of flash vac


In [105]:
m_mashnew4 = flashvac_m_mashout + m_h2omash_out4  # total flowrate (mash + liq water) coming out of flash vacuum
W_mash4 = flashvac_m_mashout/m_mashnew4
W_h2o4 = m_h2omash_out4/m_mashnew4

print(f"W_mash4 = {W_mash4 :.6f} new weight % of corn solids in mash out of flash vac")
print(f"W_h2o4 = {W_h2o4 :.6f} new weight % of water in mash out of flash vac")
print(f"m_mashnew4 = {m_mashnew4 :.6f} lbm/hr = flowrate of mash and liq water out of flash vac")

W_mash4 = 0.291755 new weight % of corn solids in mash out of flash vac
W_h2o4 = 0.708245 new weight % of water in mash out of flash vac
m_mashnew4 = 475055.638352 lbm/hr = flowrate of mash and liq water out of flash vac


### ANSWERS To Question 9 ###

** min_pressure cooking vessel must operate = 6.2464 bar ** \
** flash_pressure to provide resultant temp of 145 F = 0.2263 bar ** \
** m_h2ovaporout3 = 42448.543516 lbm/hr = flowrate of vapor out of first flash ** \** m_h2ovaporout4 = 54871.251358 lbm/hr = flowrate of vapor out of flash vac**


In [110]:
# Question 10

In [112]:
# going back to heat exchanger heating mash to 230 F

print(f"W_mash1 = {W_mash1 :.6f} new weight % of corn solids in mash out of precook vessel")
print(f"W_h2o1 = {W_h2o1 :.6f} new weight % of water in mash out of precook vessel")
print(f"m_mashnew1 = {m_mashnew1 :.6f} lbm/hr = flowrate of mash and water out of precook vessel")


W_mash1 = 0.266284 new weight % of corn solids in mash out of precook vessel
W_h2o1 = 0.733716 new weight % of water in mash out of precook vessel
m_mashnew1 = 520496.427040 lbm/hr = flowrate of mash and water out of precook vessel


In [113]:
tin_mashHE = 62.78 # in C, or 145 F, inlet temp of mash going into Heat Exchanger
tout_mashHE = 110 # in C or 230 F, outlet temp of mash
tin_out_steamHE = 121 # in C Tsat of water in 15 psig sat steam

HE_m_mashin = m_mashnew1 * W_mash1 # mass flowrate of mash solids into Heat Exchanger
HE_m_h2oin = m_mashnew1 * W_h2o1 # mass flowrate of water into Heat Exchanger
HE_m_mashout = HE_m_mashin # mass flowrate of mash solids OUT of Heat Exchanger

print(f"HE_m_h2oin = {HE_m_h2oin :.6f} lbm/hr = flowrate of water coming in to heat exchanger")
print(f"HE_m_mashin = {HE_m_mashin :.6f} lbm/hr = flowrate of mash solids coming in to heat exchanger")

HE_m_h2oin = 381896.427040 lbm/hr = flowrate of water coming in to heat exchanger
HE_m_mashin = 138600.000000 lbm/hr = flowrate of mash solids coming in to heat exchanger


In [114]:
# calculating delta hs and hs

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmashHE = quad(Cpmash, tin_mashHE, tout_mashHE)[0]
delta_hmash_jPgHE = delta_hmashHE/MWh2o
delta_hmash_jPlbmHE = delta_hmash_jPgHE/lbmPg

print(f"delta_hmashHE = {delta_hmash_jPlbmHE :.6f} J/lbm = enthalpy change of mash in Heat Exchanger")

delta_hmashHE = 89690.802989 J/lbm = enthalpy change of mash in Heat Exchanger


In [115]:
def Cph2o(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hh2oHE = quad(Cph2o, tin_mashHE, tout_mashHE)[0]
delta_hh2o_jPgHE = delta_hh2oHE/MWh2o
delta_hh2o_jPlbmHE = delta_hh2o_jPgHE/lbmPg

print(f"delta_hh2oHE = {delta_hh2o_jPlbmHE :.6f} J/lbm = enthalpy change of h2o in mash in Heat Exchanger")

delta_hh2oHE = 89690.802989 J/lbm = enthalpy change of h2o in mash in Heat Exchanger


In [116]:
# calculating delta hs and hs

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h2w, enthalpy of water vapor coming in to Heat Exchanger
deltahvapHE = 39651 # @ 62.78 C, J/mol roughly from Dippr (positive since liquid to steam)
h_watervapor_inHE = quad(Cph2o, ref, tin_out_steamHE)[0] + deltahvapHE

# calculating h3w, enthalpy of water coming out of Heat Exchanger sat liq
h_water_outHE = quad(Cph2o, ref, tin_out_steamHE)[0]

In [122]:
h_watervapor_inHE_jPg = h_watervapor_inHE/MWh2o
h_watervapor_inHE_jPlbm = h_watervapor_inHE_jPg/lbmPg

print(f"h_watervapor_inHE_jPlbm = {h_watervapor_inHE_jPlbm :.6f} J/lbm = enthalpy watervapor coming in to Heat Exchanger")

h_water_outHE_jPg = h_water_outHE/MWh2o
h_water_outHE_jPlbm = h_water_outHE_jPg/lbmPg

print(f"h_water_outHE_jPlbm = {h_water_outHE_jPlbm :.6f} J/lbm = enthalpy water coming out of Heat Exchanger")

deltah_satstreamHE = h_water_outHE_jPlbm - h_watervapor_inHE_jPlbm
print(f"deltah_satstreamHE = {deltah_satstreamHE :.6f} J/lbm = enthalpy change of sat stream of Heat Exchanger")

h_watervapor_inHE_jPlbm = 1228690.553192 J/lbm = enthalpy watervapor coming in to Heat Exchanger
h_water_outHE_jPlbm = 229830.308379 J/lbm = enthalpy water coming out of Heat Exchanger
deltah_satstreamHE = -998860.244813 J/lbm = enthalpy change of sat stream of Heat Exchanger


In [124]:
def f_HE(a):
    m_satstream = a # unknowns, sat stream of 15 psig steam becoming sat liquid after Heat Exchanger
    
    eq1 = HE_m_mashin * delta_hmash_jPlbmHE + HE_m_h2oin * delta_hh2o_jPlbmHE + m_satstream * deltah_satstreamHE 
    return eq1

guess_HE = 500000
ansHE = fsolve(f_HE, guess_HE)
m_satstream = ansHE[0]

print(f"** m_satstream = {m_satstream :.6f} lbm/hr = flowrate of sat steam to heat mash to 230 F**")

** m_satstream = 46737.011245 lbm/hr = flowrate of sat steam to heat mash to 230 F**


### ANSWERS To Question 10 ###

** m_satstream = 46737.011245 lbm/hr = flowrate of sat steam to heat mash from precooking vessel to 230 F** \
** m_h2ovaporin2 = 51879.006185 lbm/hr = flowrate of vapor (live 150 psig steam) in to heater to raise temp to 320 F** 

** m_h2ovaporout3 = 42448.543516 lbm/hr = flowrate of vapor out of first flash at 15 psig ** 

Comparing the flowrates above, it is not feasible for the steam requirements to be met by the vapor from the flash occurring at 15 psig. There is not enough needed for the precooking vessel to be maintained at 230 F. The heater also needs 150 psig so that cannot use the vapor from the flash. If we were to still use the vapor from the flash to satisfy a portion of the requirements, it can supply 42448/46737 or around 91% of the sat steam needed. 

In [128]:
# Question 11

In [213]:
# knowns for Distillation column, ethanol concentrator
# Ethanol, water and mash going in. All ethanol goes to the top stream, all solids go to the bottom stream. mixture of water in both streams.

m_eth_inDC1 = 41102.2 # Ethanol in, lbm/hr 
m_h2o_inDC1 = 497857.8 # Water in, lbm/hr
m_mash_inDC1 = 39944.4 # mash in, lbm/hr

m_h2o_bottomsDC1 = 495870 # Water out in bottoms stream, lbm/hr
m_mash_bottomsDC1 = 39944.4 # mash out in the bottoms stream, lbm/hr

m_eth_topDC1 = 41102.2 # Ethanol out in the top stream, lbm/hr
m_h2o_topDC1 = 1987 # h2o out in the top stream, lbm/hr

t_in_DC1 = 121.11 # 250 F in C
t_bottoms_DC1 = 135 # 305 F in C
t_top_DC1 = 121.11 # 250 F in C

In [215]:
# calculating Q for reboiler

Tsat_150psig = 185 # C, Tsat of 150 psig steam for reboiler
m_150psig = 110000 # lbm/hr steam flowrate

# calculating delta hs and hs (delta hvap essentially)

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1w, enthalpy of water vapor coming in to Reboiler
deltahvap_reboiler = 35939  # @ 185 C, J/mol roughly from Dippr (positive since liquid to steam)
h_watervapor_inreboiler = quad(Cph2o, ref, Tsat_150psig)[0] + deltahvap_reboiler

# calculating h2w, enthalpy of water coming out of Heat Exchanger sat liq
h_water_outreboiler = quad(Cph2o, ref, Tsat_150psig)[0]

In [217]:
h_watervapor_inreboiler_jPg = h_watervapor_inreboiler/MWh2o
h_watervapor_inreboiler_jPlbm = h_watervapor_inreboiler_jPg/lbmPg

print(f"h_watervapor_inreboiler_jPlbm = {h_watervapor_inreboiler_jPlbm :.6f} J/lbm = enthalpy watervapor coming in to reboiler")

h_water_outreboiler_jPg = h_water_outreboiler/MWh2o
h_water_outreboiler_jPlbm = h_water_outreboiler_jPg/lbmPg

print(f"h_water_outreboiler_jPlbm = {h_water_outreboiler_jPlbm :.6f} J/lbm = enthalpy water coming out of reboiler")

deltah_satstreamreboiler = h_water_outreboiler_jPlbm - h_watervapor_inreboiler_jPlbm
print(f"deltah_satstreamreboiler = {deltah_satstreamreboiler :.6f} J/lbm = enthalpy change of sat stream of reboiler")

h_watervapor_inreboiler_jPlbm = 1256743.585111 J/lbm = enthalpy watervapor coming in to reboiler
h_water_outreboiler_jPlbm = 351393.446695 J/lbm = enthalpy water coming out of reboiler
deltah_satstreamreboiler = -905350.138416 J/lbm = enthalpy change of sat stream of reboiler


In [219]:
Qreboiler = m_150psig * deltah_satstreamreboiler

print(f"Qreboiler = {Qreboiler :.6f} J/hr = heat transferred by reboiler")

Qreboiler = -99588515225.773026 J/hr = heat transferred by reboiler


In [221]:
# calculating hs and deltahs

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmashDC1 = quad(Cpmash, t_in_DC1, t_bottoms_DC1)[0]
delta_hmashDC1_jPgHE = delta_hmashDC1/MWh2o
delta_hmashDC1_jPlbmHE = delta_hmashDC1_jPgHE/lbmPg

print(f"delta_hmashDC1_jPlbmHE = {delta_hmashDC1_jPlbmHE :.6f} J/lbm = enthalpy change of mash in Distillation Column 1(ethanol conc)")

delta_hmashDC1_jPlbmHE = 26382.999863 J/lbm = enthalpy change of mash in Distillation Column 1(ethanol conc)


In [223]:
# calculating hs and deltahs

def Cpethanol(t):
    A = 9.839E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWeth = 46.07
lbmPg = 0.002204 #conversion

delta_hethDC1 = quad(Cpethanol, t_in_DC1, t_top_DC1)[0]
delta_hethDC1_jPgHE = delta_hethDC1/MWeth
delta_hethDC1_jPlbmHE = delta_hethDC1_jPgHE/lbmPg

print(f"delta_hethDC1_jPlbmHE = {delta_hethDC1_jPlbmHE :.6f} J/lbm = enthalpy change of ethanol in Distillation Column 1(ethanol conc)")

delta_hethDC1_jPlbmHE = 0.000000 J/lbm = enthalpy change of ethanol in Distillation Column 1(ethanol conc)


In [225]:
# calculating delta hs and hs

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to DC1
h_water_inDC1 = quad(Cph2o, ref, t_in_DC1)[0]

# calculating h2, enthalpy of water coming out of DC1 at the top
h_water_outtopDC1 = quad(Cph2o, ref, t_top_DC1)[0]

# calculating h3, enthalpy of water coming out of DC1 at the bottom
h_water_outbottomDC1 = quad(Cph2o, ref, t_bottoms_DC1)[0]


In [227]:
h_water_inDC1_jPg = h_water_inDC1/MWh2o
h_water_inDC1_jPlbm = h_water_inDC1_jPg/lbmPg

print(f"h_water_inDC1_jPlbm = {h_water_inDC1_jPlbm :.6f} J/lbm = enthalpy water coming in to DC1")

h_water_outtopDC1_jPg = h_water_outtopDC1/MWh2o
h_water_outtopDC1_jPlbm = h_water_outtopDC1_jPg/lbmPg

print(f"h_water_outtopDC1_jPlbm = {h_water_outtopDC1_jPlbm :.6f} J/lbm = enthalpy water coming out of DC1 at the top")

h_water_outbottomDC1_jPg = h_water_outbottomDC1/MWh2o
h_water_outbottomDC1_jPlbm = h_water_outbottomDC1_jPg/lbmPg

print(f"h_water_outbottomDC1_jPlbm = {h_water_outbottomDC1_jPlbm :.6f} J/lbm = enthalpy water coming out of DC1 at the bottom")

h_water_inDC1_jPlbm = 230039.245023 J/lbm = enthalpy water coming in to DC1
h_water_outtopDC1_jPlbm = 230039.245023 J/lbm = enthalpy water coming out of DC1 at the top
h_water_outbottomDC1_jPlbm = 256422.244885 J/lbm = enthalpy water coming out of DC1 at the bottom


In [229]:
def f_DC1(a):
    Qcondenser = a # unknowns, heat transferred in condenser
    
    eq1 = m_mash_bottomsDC1*delta_hmashDC1_jPlbmHE + m_eth_topDC1*delta_hethDC1_jPlbmHE + m_h2o_bottomsDC1*h_water_outbottomDC1_jPlbm + m_h2o_topDC1*h_water_outtopDC1_jPlbm - m_h2o_inDC1*h_water_inDC1_jPlbm + Qcondenser + Qreboiler 
    return eq1

guess_DC1 = 500000
ansDC1 = fsolve(f_DC1, guess_DC1)
QcondenserDC1 = ansDC1[0]

print(f"** QcondenserDC1 = {QcondenserDC1 :.6f} J/hr = heat transferred in condenser in DC1**")

** QcondenserDC1 = 85452308015.559418 J/hr = heat transferred in condenser in DC1**


In [231]:
# calculating delta h water in condenser in DC1 (liquid going from 85 to 110 F)

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

t_incooling_water =  29.44 # 85 F in C
t_outcooling_water = 43.33 # 110 F in C

deltah_condenser_DC1 = quad(Cph2o, t_incooling_water, t_outcooling_water)[0] 

deltah_condenser_DC1_jPg = deltah_condenser_DC1/MWh2o
deltah_condenser_DC1_jPlbm = deltah_condenser_DC1_jPg/lbmPg

print(f"deltah_condenser_DC1_jPlbm = {deltah_condenser_DC1_jPlbm :.6f} J/lbm = enthalpy change of water supplied to condenser in DC1")

deltah_condenser_DC1_jPlbm = 26382.999863 J/lbm = enthalpy change of water supplied to condenser in DC1


In [233]:
m_h2o_condenser_DC1 = QcondenserDC1 / deltah_condenser_DC1_jPlbm

print(f"** m_h2o_condenser_DC1 = {m_h2o_condenser_DC1 :.6f} lbm/hr = flowrate of water supplied to condenser in DC1**")

** m_h2o_condenser_DC1 = 3238915.531229 lbm/hr = flowrate of water supplied to condenser in DC1**


In [286]:
# solving around condenser of azeotrope column
# vapor pentane, ethanol, water coming in and all condensed to liquid

m_pent_in_condAC = 179386.53 # lbm/hr
m_ethanol_in_condAC = 15515.94 # lbm/hr
m_h2o_in_condAC = 12525.28 # lbm/hr

In [288]:
Tsat_condAC = 66.11 # 151 F in C

#deltahvaps in positive since they are going from steam to liquid

deltahvap_pent_condAC = -23849 # J/mol
deltahvap_ethanol_condAC = -40048 # J/mol
deltahvap_h2o_condAC = -42249 # J/mol

MWpent = 72.15 # g/mol
MWethanol = 46.07 # g/mol

In [290]:

deltahvap_pent_condAC_jPg = deltahvap_pent_condAC/MWpent
deltahvap_pent_condAC_jPlbm = deltahvap_pent_condAC_jPg/lbmPg

print(f"deltahvap_pent_condAC_jPlbm = {deltahvap_pent_condAC_jPlbm :.6f} J/lbm = enthalpy pent coming in to condenser for Azeotrope Column")

deltahvap_ethanol_condAC_jPg = deltahvap_ethanol_condAC/MWethanol
deltahvap_ethanol_condAC_jPlbm = deltahvap_ethanol_condAC_jPg/lbmPg

print(f"deltahvap_ethanol_condAC_jPlbm = {deltahvap_ethanol_condAC_jPlbm :.6f} J/lbm = enthalpy ethanol coming in to condenser for Azeotrope Column")

deltahvap_h2o_condAC_jPg = deltahvap_h2o_condAC/MWh2o
deltahvap_h2o_condAC_jPlbm = deltahvap_h2o_condAC_jPg/lbmPg

print(f"deltahvap_h2o_condAC_jPlbm = {deltahvap_h2o_condAC_jPlbm :.6f} J/lbm = enthalpy water coming out of DC1 at the bottom")

deltahvap_pent_condAC_jPlbm = -149976.166310 J/lbm = enthalpy pent coming in to condenser for Azeotrope Column
deltahvap_ethanol_condAC_jPlbm = -394412.826374 J/lbm = enthalpy ethanol coming in to condenser for Azeotrope Column
deltahvap_h2o_condAC_jPlbm = -1064307.242771 J/lbm = enthalpy water coming out of DC1 at the bottom


In [308]:
Qcondenser_AC = m_pent_in_condAC*deltahvap_pent_condAC_jPlbm + m_ethanol_in_condAC*deltahvap_ethanol_condAC_jPlbm + m_h2o_in_condAC*deltahvap_h2o_condAC_jPlbm 

print(f"Qcondenser_AC = {Qcondenser_AC :.6f} J/hr = heat transferred by stream in condenser of Azeotrope Column")

Qcondenser_AC = -46354136028.062134 J/hr = heat transferred by stream in condenser of Azeotrope Column


In [310]:
# calculating delta h water in condenser in Azeotrope Column (liquid going from 85 to 110 F)

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

t_incooling_water =  29.44 # 85 F in C
t_outcooling_water = 43.33 # 110 F in C

deltah_condenser_AC = quad(Cph2o, t_incooling_water, t_outcooling_water)[0] 

deltah_condenser_AC_jPg = deltah_condenser_AC/MWh2o
deltah_condenser_AC_jPlbm = deltah_condenser_AC_jPg/lbmPg

print(f"deltah_condenser_AC_jPlbm = {deltah_condenser_AC_jPlbm :.6f} J/lbm = enthalpy change of water supplied to condenser in Azeotrope Column")

deltah_condenser_AC_jPlbm = 26382.999863 J/lbm = enthalpy change of water supplied to condenser in Azeotrope Column


In [312]:
m_h2o_condenser_AC = -Qcondenser_AC / deltah_condenser_AC_jPlbm 

print(f"** m_h2o_condenser_AC = {m_h2o_condenser_AC :.6f} lbm/hr = flowrate of water supplied to condenser in AC**")

** m_h2o_condenser_AC = 1756969.877170 lbm/hr = flowrate of water supplied to condenser in AC**


In [314]:
# solving around Stripper, need to get actual numbers

Tsat_top = 97.78 # 208 F in C

m_h2o_topDC1 = 1987 # h2o out in the top stream of DC1, lbm/hr
m_h2o_inAC = m_h2o_topDC1
m_h2o_bottomproduct_AC = 341.7122 # lbm/hr

m_h2o_wasteout_Str = m_h2o_inAC - m_h2o_bottomproduct_AC # h2o waste in bottom stream of Stripper, lbm/hr

m_h2o_inStr = 2776.156538 # lbm/hr, water into stripper
m_ethanol_inStr = 2366.47769 # lbm/hr, ethanol into stripper 

m_h2o_topStr = m_h2o_inStr - m_h2o_wasteout_Str # h2o coming out in the boilup stream of stripper

print(f"m_h2o_topStr = {m_h2o_topStr :.4f} lbm/hr, h2o coming out in the boilup stream of stripper")
print(f"m_ethanol_inStr = {m_ethanol_inStr :.4f} lbm/hr, ethanol coming out in the boilup stream of stripper")

#deltahvaps in positive since they are going from steam to liquid
deltahvap_ethanol_condStr = -37764 # J/mol
deltahvap_h2o_condStr = -40800 # J/mol

m_h2o_topStr = 1130.8687 lbm/hr, h2o coming out in the boilup stream of stripper
m_ethanol_inStr = 2366.4777 lbm/hr, ethanol coming out in the boilup stream of stripper


In [316]:
deltahvap_ethanol_condStr_jPg = deltahvap_ethanol_condStr/MWethanol
deltahvap_ethanol_condStr_jPlbm = deltahvap_ethanol_condStr_jPg/lbmPg

print(f"deltahvap_ethanol_condStr_jPlbm = {deltahvap_ethanol_condStr_jPlbm :.6f} J/lbm = enthalpy ethanol coming in to condenser for Stripper Column")

deltahvap_h2o_condStr_jPg = deltahvap_h2o_condStr/MWh2o
deltahvap_h2o_condStr_jPlbm = deltahvap_h2o_condStr_jPg/lbmPg

print(f"deltahvap_h2o_condStr_jPlbm = {deltahvap_h2o_condStr_jPlbm :.6f} J/lbm = enthalpy water coming in to condenser for Stripper Column")

deltahvap_ethanol_condStr_jPlbm = -371918.846764 J/lbm = enthalpy ethanol coming in to condenser for Stripper Column
deltahvap_h2o_condStr_jPlbm = -1027805.048760 J/lbm = enthalpy water coming in to condenser for Stripper Column


In [318]:
Qcondenser_Stripper = m_h2o_topStr * deltahvap_h2o_condStr_jPlbm + m_ethanol_inStr * deltahvap_ethanol_condStr_jPlbm 

print(f"Qcondenser_Stripper = {Qcondenser_Stripper :.6f} J/hr = heat transferred by stream in condenser of Stripper")

Qcondenser_Stripper = -2042450251.758530 J/hr = heat transferred by stream in condenser of Stripper


In [324]:
# calculating delta h water in condenser in Stripper (liquid going from 85 to 110 F)

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

t_incooling_water =  29.44 # 85 F in C
t_outcooling_water = 43.33 # 110 F in C

deltah_condenser_Str = quad(Cph2o, t_incooling_water, t_outcooling_water)[0] 

deltah_condenser_Str_jPg = deltah_condenser_Str/MWh2o
deltah_condenser_Str_jPlbm = deltah_condenser_Str_jPg/lbmPg

print(f"deltah_condenser_Str_jPlbm = {deltah_condenser_Str_jPlbm :.6f} J/lbm = enthalpy change of water supplied to condenser in Stripper")

deltah_condenser_Str_jPlbm = 26382.999863 J/lbm = enthalpy change of water supplied to condenser in Stripper


In [326]:
m_h2o_condenser_Str = -Qcondenser_Stripper  / deltah_condenser_Str_jPlbm  

print(f"** m_h2o_condenser_Str = {m_h2o_condenser_Str :.6f} lbm/hr = flowrate of water supplied to condenser in Stripper**")

** m_h2o_condenser_Str = 77415.391062 lbm/hr = flowrate of water supplied to condenser in Stripper**


### Question 11 ###

We calculated the total mass flow rates of the water necessary to cool each of the condensers from each distillation column and the stripper. We did a total mass balance on the whole system starting from the Azeotrope Column to the stripper. After finding the pertinent mass flow rates, we solved for each Q. In the first distillation column, we solved for total heat provided by steam, then did an energy balance to find the remaining heat necessary to cool the distillate in condenser 1. That number came out to deltah_condenser_DC1_jPlbm = 26382.999863 J/lbm = enthalpy change of water supplied to condenser in DC1. 
We then took this number and divided it by the heat capacity of water from 85F to 110F which gave us a total mass flow rate of water necessary at ** m_h2o_condenser_DC1 = 3238915.531229 lbm/hr = flowrate of water supplied to condenser in DC1**.

We did a similar process in Condensers 2 and three, using the deltahVaporization for each component to find each Q.
 m_h2o_condenser_AC = 1756969.877170 lbm/hr = flowrate of water supplied to condenser in AC
 m_h2o_condenser_Str = 77415.391062 lbm/hr = flowrate of water supplied to condenser in Stripper**

